# 1. Thư viện

In [1]:
import os
import json

from dotenv import load_dotenv
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore, RetrievalMode

# 2. Model embedding

## 2.1. Dùng model embedding gpt4all từ langchain

In [2]:
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

model_Dense_Embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs,
    )

# 3. Tạo Collection

In [3]:
load_dotenv()

URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")

In [4]:
url = URL_QDRANT
api_key = API_QDRANT

## 3.1. Load file data

In [5]:
with open(r'data\tinh_huong\json\page_content.json', 'r', encoding='utf-8') as f:
    page_content_list = json.load(f)

with open(r'data\tinh_huong\json\metadata.json', 'r', encoding='utf-8') as f:
    metadata_list = json.load(f)

In [6]:
documents = []
for page_content, metadata in zip(page_content_list, metadata_list):
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

In [7]:
# In ra 3 document đầu tiên
for doc in documents[:3]:
    print("Page Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 50)  # Để phân cách giữa các document

Page Content: Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?
Metadata: {'key_word': ['người', 'dân', 'tộc', 'thiểu', 'số', 'xã', 'nông', 'thôn', 'mới', 'có', 'được', 'hỗ', 'trợ', 'bhyt'], 'context': 'Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?\nBộ Y tế trả lời vấn đề này như sau:Ngày 19/10/2023, Chính phủ ban hành Nghị định số75/2023/NĐ-CPsửa đổi, bổ sung một số điều của Nghị định số 146/2023/NĐ-CP ngày 17/10/2018 của Chính phủ quy định chi tiết và hướng dẫn biện pháp thi hành một số điều của Luật BHYT, trong đó có quy định:Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn

In [8]:
# # Đường dẫn tới file JSON
# file_path = r'data\tinh_huong\json\data_context.json'

# # Mở và load file JSON
# with open(file_path, 'r', encoding='utf-8') as f:
#     data = json.load(f)

In [9]:
# # In ra 3 đối tượng đầu tiên
# for item in data[:3]:
#     print(item['question'],"\n")
#     print(item['answer'])
#     print('-' * 40)  # Đường phân cách giữa các đối tượng

## 3.2. Tạo các documents từ data

In [10]:
# # Tạo danh sách các Document
# documents = [Document(
#     page_content = item['question'],
#     metadata = {
#         "context": item['answer'],
#         "key_word": item['question'].lower().split()  # Tách từ và chuyển thành lowercase
#         }
# ) for item in data]

In [11]:
# # In ra thông tin các Document để kiểm tra
# for doc in documents[:3]:  # In 3 đối tượng đầu tiên để kiểm tra
#     print('Page Content:', doc.page_content,'\n')
#     print('Metadata \"context\":', doc.metadata['context'],"\n")
#     print('Metadata \"key_word\":', doc.metadata['key_word'])
#     print('-' * 40)

## 3.3. Tạo collection

In [12]:
qdrant = QdrantVectorStore.from_documents(
    documents = documents,
    embedding = model_Dense_Embedding,
    url = url,
    api_key = api_key,
    prefer_grpc=True,
    collection_name = "dense_TinhHuong",
    retrieval_mode=RetrievalMode.DENSE,
	metadata_payload_key="metadata"
)